In [4]:
import sqlite3
import pandas as pd

In [5]:
_DB_PATH = "./db.sqlite"

In [6]:
def db_con() -> sqlite3.Connection:
    con: sqlite3.Connection = sqlite3.connect(_DB_PATH)
    con.row_factory = sqlite3.Row
    return con


def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [8]:
query("select * from keywords limit 3")

,id,chunk_id,word,count
0,1,1,*,12
1,2,1,福岡,1
2,3,1,中央,1


In [23]:
query("select count(distinct chunk_id) from keywords")

,count(distinct chunk_id)
0,45


In [26]:
query("""
select
  word,
  count(distinct chunk_id) as document_count
from
  keywords
group by
  word      
""")

,word,document_count
0,*,43
1,あずさ,1
2,お客,7
3,お客様,4
4,お金,5
...,...,...
413,高速,1
414,高雄,1
415,鬼木,1
416,黒田,1


In [24]:
query("""
with document_length as (
    select
      chunk_id,
      sum(count) as length
    from
      keywords
    group by
      chunk_id
), term_frequency as (
    select
      k.chunk_id,
      k.word,
      k.count,
      dl.length
    from
      keywords k
        inner join
      document_length dl
        on k.chunk_id = dl.chunk_id
)
select
  tf.chunk_id,
  tf.word,
  1.0 * tf.count /
    tf.length as term_frequency,
  1.0 * dc.document_count / 
    (select count(distinct chunk_id) from keywords) as document_frequency
from
  term_frequency tf
    inner join
  document_count dc
    on tf.word = dc.word
""")

,chunk_id,word,term_frequency,document_frequency
0,1,*,0.206897,0.955556
1,1,ふくおか,0.017241,0.333333
2,1,オンライン,0.017241,0.022222
3,1,セミナー,0.051724,0.044444
4,1,ヒョウ,0.017241,0.044444
...,...,...,...,...
869,44,速度,0.000157,0.022222
870,44,長期,0.000157,0.022222
871,44,長距離,0.000157,0.022222
872,44,高速,0.000157,0.022222
